<a href="https://colab.research.google.com/github/AnkitRajSri/Impact-of-social-distancing-and-isolation-on-mental-health/blob/master/Florida_Hospitals_ER_Services_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing the libraries

In [0]:
# !pip install tabula-py
import pandas as pd
import numpy as np
import tabula

### Data Preprocessing

##### Importing the data

In [97]:
pdf_data = tabula.read_pdf('Hospital_ER_Services.pdf', multiple_tables=False, pages='all')
hospital_data = pd.DataFrame(pdf_data[0])
hospital_data.shape

(336, 40)

##### Data Cleaning and Manipulation

In [98]:
# hospital_data.drop(['Unnamed: 39'], axis=1, inplace=True)
hospital_data.columns = ['License Number', 'Provider Name', 'County', 'No Emergency Room Services', 'Dedicated Emergency Department', 'Anesthesia', 'Burn', 'Cardiology', 'Cardiovascular Surgery', 'Colon & Rectal Surgery', 'Emergency Medicine', 'Endocrinology', 'Gastroenterology',	'General Surgery', 'Gynecology',	'Hematology', 'Hyperbaric Medicine', 'Internal Medicine',	'Nephrology', 'Neurology',	'Neurosurgery',	'Obstetrics', 'Ophthalmology',	'Oral/Maxillo-Facial Surgery',	'Orthopedics',	'Otolaryngology', 'Plastic Surgery', 'Podiatry', 'Psychiatry', 'Pulmonary Medicine', 'Radiology',	'Thoracic Surgery', 'Urology', 'Vascular Surgery', 'Level 1 Trauma Center', 'Level 2 Trauma Center', 'Pediatric Trauma Center', 'Provisional Level 1 Trauma Center', 'Provisional Level 2 Trauma Center', 'Provisional Pediatric Trauma Center']
hospital_data.head()

,License Number,Provider Name,County,No Emergency Room Services,Dedicated Emergency Department,Anesthesia,Burn,Cardiology,Cardiovascular Surgery,Colon & Rectal Surgery,Emergency Medicine,Endocrinology,Gastroenterology,General Surgery,Gynecology,Hematology,Hyperbaric Medicine,Internal Medicine,Nephrology,Neurology,Neurosurgery,Obstetrics,Ophthalmology,Oral/Maxillo-Facial Surgery,Orthopedics,Otolaryngology,Plastic Surgery,Podiatry,Psychiatry,Pulmonary Medicine,Radiology,Thoracic Surgery,Urology,Vascular Surgery,Level 1 Trauma Center,Level 2 Trauma Center,Pediatric Trauma Center,Provisional Level 1 Trauma Center,Provisional Level 2 Trauma Center,Provisional Pediatric Trauma Center
0,4247,NORTH FLORIDA REGIONAL\rMEDICAL CENTER,ALACHUA,NaN,X,X,NaN,X,X,NaN,X,NX,EX,eXr,eXn,Xy,o,mX,Xer,iXc,sX,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN,NaN,NaN,NaN,NaN,NaN
1,4489,SELECT SPECIALTY\rHOSPITAL GAINESVILLE,ALACHUA,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,An,st,es,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4529,UF HEALTH REHAB\rHOSPITAL,ALACHUA,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ca\rCa,di\rdi,lo\rva,y\rcu,ar,ur,er,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4286,UF HEALTH SHANDS\rHOSPITAL,ALACHUA,NaN,X,X,X,X,X,X,X,Co\rX\rE,on\rX\rer,&\rX\ren,ec\rX\ry,al\rX\re,ur\rici,e\rX\re,y\rX,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN,NaN,NaN,NaN,NaN
4,4286,UF HEALTH SHANDS\rPSYCHIATRIC HOSPITAL,ALACHUA,X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,En\rGa,oc\rtr,in\ren,lo\rer,y\rlo,y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
hospital_data = hospital_data[hospital_data['License Number']!='License\rNumber']
hospital_data.shape

(307, 40)

In [100]:
hospital_data = hospital_data.fillna(0)
hospital_data.head()

,License Number,Provider Name,County,No Emergency Room Services,Dedicated Emergency Department,Anesthesia,Burn,Cardiology,Cardiovascular Surgery,Colon & Rectal Surgery,Emergency Medicine,Endocrinology,Gastroenterology,General Surgery,Gynecology,Hematology,Hyperbaric Medicine,Internal Medicine,Nephrology,Neurology,Neurosurgery,Obstetrics,Ophthalmology,Oral/Maxillo-Facial Surgery,Orthopedics,Otolaryngology,Plastic Surgery,Podiatry,Psychiatry,Pulmonary Medicine,Radiology,Thoracic Surgery,Urology,Vascular Surgery,Level 1 Trauma Center,Level 2 Trauma Center,Pediatric Trauma Center,Provisional Level 1 Trauma Center,Provisional Level 2 Trauma Center,Provisional Pediatric Trauma Center
0,4247,NORTH FLORIDA REGIONAL\rMEDICAL CENTER,ALACHUA,0,X,X,0,X,X,0,X,NX,EX,eXr,eXn,Xy,o,mX,Xer,iXc,sX,X,X,X,X,X,X,X,X,X,X,X,X,X,0,0,0,0.0,0.0,0.0
1,4489,SELECT SPECIALTY\rHOSPITAL GAINESVILLE,ALACHUA,X,0,0,0,0,0,0,0,An,st,es,a,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0
2,4529,UF HEALTH REHAB\rHOSPITAL,ALACHUA,X,0,0,0,0,0,0,0,Ca\rCa,di\rdi,lo\rva,y\rcu,ar,ur,er,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0
3,4286,UF HEALTH SHANDS\rHOSPITAL,ALACHUA,0,X,X,X,X,X,X,X,Co\rX\rE,on\rX\rer,&\rX\ren,ec\rX\ry,al\rX\re,ur\rici,e\rX\re,y\rX,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,0,0,0.0,0.0,0.0
4,4286,UF HEALTH SHANDS\rPSYCHIATRIC HOSPITAL,ALACHUA,X,0,0,0,0,0,0,0,En\rGa,oc\rtr,in\ren,lo\rer,y\rlo,y,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0


In [0]:
for col in hospital_data.columns:
  if col in ['License Number', 'Provider Name', 'County']:
    continue
  else:
    hospital_data[col] = [1 if 'X' in str(x) else 0 for x in hospital_data[col]]

In [102]:
hospital_data.head(20)

,License Number,Provider Name,County,No Emergency Room Services,Dedicated Emergency Department,Anesthesia,Burn,Cardiology,Cardiovascular Surgery,Colon & Rectal Surgery,Emergency Medicine,Endocrinology,Gastroenterology,General Surgery,Gynecology,Hematology,Hyperbaric Medicine,Internal Medicine,Nephrology,Neurology,Neurosurgery,Obstetrics,Ophthalmology,Oral/Maxillo-Facial Surgery,Orthopedics,Otolaryngology,Plastic Surgery,Podiatry,Psychiatry,Pulmonary Medicine,Radiology,Thoracic Surgery,Urology,Vascular Surgery,Level 1 Trauma Center,Level 2 Trauma Center,Pediatric Trauma Center,Provisional Level 1 Trauma Center,Provisional Level 2 Trauma Center,Provisional Pediatric Trauma Center
0,4247,NORTH FLORIDA REGIONAL\rMEDICAL CENTER,ALACHUA,0,1,1,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0
1,4489,SELECT SPECIALTY\rHOSPITAL GAINESVILLE,ALACHUA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4529,UF HEALTH REHAB\rHOSPITAL,ALACHUA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4286,UF HEALTH SHANDS\rHOSPITAL,ALACHUA,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0
4,4286,UF HEALTH SHANDS\rPSYCHIATRIC HOSPITAL,ALACHUA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,4152,ED FRASER MEMORIAL\rHOSPITAL,BAKER,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,4004,NORTHEAST FLORIDA\rSTATE HOSPITAL,BAKER,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,3982,ASCENSION SACRED HEART\rBAY,BAY,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,1,0,0,0,0
8,4500,EMERALD COAST\rBEHAVIORAL HOSPITAL,BAY,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,4458,ENCOMPASS HEALTH\rREHABILITATION HOSPITAL\rOF ...,BAY,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
import re

hospital_data['Provider Name'] = hospital_data['Provider Name'].apply(lambda x : re.sub('\r', ' ', x))

In [104]:
hospital_data.head()

,License Number,Provider Name,County,No Emergency Room Services,Dedicated Emergency Department,Anesthesia,Burn,Cardiology,Cardiovascular Surgery,Colon & Rectal Surgery,Emergency Medicine,Endocrinology,Gastroenterology,General Surgery,Gynecology,Hematology,Hyperbaric Medicine,Internal Medicine,Nephrology,Neurology,Neurosurgery,Obstetrics,Ophthalmology,Oral/Maxillo-Facial Surgery,Orthopedics,Otolaryngology,Plastic Surgery,Podiatry,Psychiatry,Pulmonary Medicine,Radiology,Thoracic Surgery,Urology,Vascular Surgery,Level 1 Trauma Center,Level 2 Trauma Center,Pediatric Trauma Center,Provisional Level 1 Trauma Center,Provisional Level 2 Trauma Center,Provisional Pediatric Trauma Center
0,4247,NORTH FLORIDA REGIONAL MEDICAL CENTER,ALACHUA,0,1,1,0,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0
1,4489,SELECT SPECIALTY HOSPITAL GAINESVILLE,ALACHUA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4529,UF HEALTH REHAB HOSPITAL,ALACHUA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4286,UF HEALTH SHANDS HOSPITAL,ALACHUA,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0
4,4286,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,ALACHUA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [105]:
psychiatric_facilities = pd.DataFrame(hospital_data['Psychiatry'].value_counts()).reset_index()
psychiatric_facilities

,index,Psychiatry
0,1,156
1,0,151


In [106]:
psychiatric_facilities.columns = ['Psychiatric Facility', 'Count']
psychiatric_facilities.loc[psychiatric_facilities['Psychiatric Facility']==1, 'Psychiatric Facility'] = 'Available'
psychiatric_facilities.loc[psychiatric_facilities['Psychiatric Facility']==0, 'Psychiatric Facility'] = 'Not Available'
psychiatric_facilities

,Psychiatric Facility,Count
0,Available,156
1,Not Available,151


In [107]:
import plotly.express as px

fig = px.pie(psychiatric_facilities, values='Count', names='Psychiatric Facility', title='% Availability of Psychiatric Facilities')
fig.show()

In [108]:
county_hospital_data = pd.DataFrame(hospital_data['County'].value_counts()).reset_index()
county_hospital_data.columns = ['County', 'Number of hospitals']
county_hospital_data.head()

,County,Number of hospitals
0,MIAMI-DADE,34
1,BROWARD,23
2,PALM BEACH,17
3,PINELLAS,16
4,ORANGE,15


In [109]:
fig = px.bar(county_hospital_data, x='County', y='Number of hospitals', title='Number of hospitals in each county')
fig.show()

In [122]:
psychiatric_facilities_data = hospital_data[['License Number', 'Provider Name', 'County', 'Psychiatry']]
psychiatric_facilities_data.head()

,License Number,Provider Name,County,Psychiatry
0,4247,NORTH FLORIDA REGIONAL MEDICAL CENTER,ALACHUA,1
1,4489,SELECT SPECIALTY HOSPITAL GAINESVILLE,ALACHUA,0
2,4529,UF HEALTH REHAB HOSPITAL,ALACHUA,0
3,4286,UF HEALTH SHANDS HOSPITAL,ALACHUA,1
4,4286,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,ALACHUA,0


In [123]:
psychiatric_dummies = pd.get_dummies(psychiatric_facilities_data['Psychiatry'])
psychiatric_dummies.columns = ['Psychiatry Not Available', 'Psychiatry Available']
psychiatric_dummies.head()

,Psychiatry Not Available,Psychiatry Available
0,0,1
1,1,0
2,1,0
3,0,1
4,1,0


In [124]:
psychiatric_facilities_data = pd.concat((psychiatric_facilities_data.drop(['Psychiatry'], axis=1`), psychiatric_dummies), axis=1)
psychiatric_facilities_data.head()

,License Number,Provider Name,County,Psychiatry Not Available,Psychiatry Available
0,4247,NORTH FLORIDA REGIONAL MEDICAL CENTER,ALACHUA,0,1
1,4489,SELECT SPECIALTY HOSPITAL GAINESVILLE,ALACHUA,1,0
2,4529,UF HEALTH REHAB HOSPITAL,ALACHUA,1,0
3,4286,UF HEALTH SHANDS HOSPITAL,ALACHUA,0,1
4,4286,UF HEALTH SHANDS PSYCHIATRIC HOSPITAL,ALACHUA,1,0


In [127]:
county_psychiatry_data = pd.DataFrame(psychiatric_facilities_data.loc[psychiatric_facilities_data['Psychiatry Available']==1, 'County'].value_counts()).reset_index()
county_psychiatry_data.columns = ['County', 'Number of hospitals']
county_psychiatry_data.head()

,County,Number of hospitals
0,MIAMI-DADE,18
1,BROWARD,18
2,PALM BEACH,10
3,PINELLAS,10
4,DUVAL,9


In [128]:
fig = px.bar(county_psychiatry_data, x='County', y='Number of hospitals', title='Number of psychiatric hospitals in each county')
fig.show()

In [134]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Bar(x=county_hospital_data.head(4)['County'], y=county_hospital_data.head(4)['Number of hospitals'], name='Total number of hospitals'),
                      go.Bar(x=county_psychiatry_data.head(4)['County'], y=county_psychiatry_data.head(4)['Number of hospitals'], name='Number of hospitals having psychiatric facilities')],)

fig.show()